<a href="https://colab.research.google.com/github/alecmeade/cs242_final_alec_daniel/blob/master/Dan_Alec_Middle_Out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Imports 
import tensorflow as tf
from tensorflow import keras
import numpy as np
import glob
from skimage import io
from skimage.transform import rescale
import matplotlib.pyplot as plt


Video samples: https://media.xiph.org/


In [0]:
#@title Data Ingest functions

#this annoyingly doesnt work right now
frame = wget.download("http://192.168.254.64:8000/trees/frame_00001.png")

In [0]:
#@title Error Functions

#Calculate the psnr error between the golden frames and the predicted frames.
def EvaluatePSNR(golden_frames, predicted_frames):
  '''
  golden_frames: A tf.tensor representing the set of original sequential frames.
  predicted_frames: A tf.tensor reprsenting the set of predicted frames. 

  return: The psnr between the two transformed in some way to fit with the 
  understanding of error.
  '''
  psnr = tf.image.psnr(golden_frames, predicted_frames, max_val=1.0)
  #TODO(scale the psnr)
  return 50 - psnr

#Calculate the ssim error between the golden frames and the predicted frames.
def EvaluateSSIM(golden_frames, predicted_frames):
  '''
  golden_frames: A tf.tensor representing the set of original sequential frames.
  predicted_frames: A tf.tensor reprsenting the set of predicted frames. 

  return: The sse between the two transformed in some way to fit with the 
  understanding of error.
  '''
  # TODO(Determine if parameters are accurate)
  return tf.image.ssim(golden_frames, predicted_frames)

#Calculate the ssim error between the golden frames and the predicted frames.
def EvaluateSSIM(golden_frames, predicted_frames):
  '''
  golden_frames: A tf.tensor representing the set of original sequential frames.
  predicted_frames: A tf.tensor reprsenting the set of predicted frames. 

  return: The sse between the two transformed in some way to fit with the 
  understanding of error.
  '''
  # TODO(Determine if parameters are accurate)
  return tf.image.ssim(golden_frames, predicted_frames)


#tf.image.ssim_multiscale(
#    img1, img2, max_val, power_factors=_MSSSIM_WEIGHTS, filter_size=11,
#    filter_sigma=1.5, k1=0.01, k2=0.03
#)

#Calculate the vgg error between the golden frames and the predicted frames.
def EvaluateVGG(golden_frames, predicted_frames):
  '''
  golden_frames: A tf.tensor representing the set of original sequential frames.
  predicted_frames: A tf.tensor reprsenting the set of predicted frames. 

  return: The vgg between the two transformed in some way to fit with the 
  understanding of error.
  '''
  raise NotImplementedError

In [0]:
#@title Network Building Functions

#Create a unet with no skip level connections. The number of filters is constent
#across the entire network.
def BuildModel(input_shape, depth):
  input_layer = keras.layers.Input(shape=input_shape)

  # Make Encoder
  encoder_last_layer = input_layer
  filter_num = input_shape[2]
  for i in range(depth):
    encoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(encoder_last_layer)
    encoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(encoder_last_layer)
    encoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(encoder_last_layer)
    encoder_last_layer = keras.layers.MaxPool2D()(encoder_last_layer)
  # Add the last three convolutional layers
  encoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(encoder_last_layer)
  encoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(encoder_last_layer)
  encoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(encoder_last_layer)

  decoder_input = keras.layers.Conv2DTranspose(filter_num, 3, padding='same', strides=2, activation=tf.nn.relu)(encoder_last_layer)
  decoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(decoder_input)
  decoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(decoder_last_layer)
  for i in range(depth - 1):
    decoder_last_layer = keras.layers.Conv2DTranspose(filter_num, 3, padding='same', strides=2, activation=tf.nn.relu)(decoder_last_layer)
    decoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(decoder_last_layer)
    decoder_last_layer = keras.layers.Conv2D(filter_num, 3, padding='same', activation=tf.nn.relu)(decoder_last_layer)
  return (input_layer, encoder_last_layer, decoder_input, decoder_last_layer)

In [0]:
#@title Plot Input, Output Frames, and the diff
def PlotFrames(golden_frames, predicted_frames):
  frame_shape = golden_frames.shape
  num_images = int(frame_shape[2] / 3)

  fig, axes = plt.subplots(num_images, 3, figsize=(40, 40))
  for i in range(num_images):
    golden_image = golden_frames[:,:,(3 * i):(3 * i) + 3]
    axes[i, 0].imshow(golden_image)
    predicted_image = predicted_frames[:,:,(3 * i):(3 * i) + 3]
    axes[i, 1].imshow(predicted_image)
    diff_image = tf.math.abs(golden_image - predicted_image)
    axes[i, 2].imshow(diff_image)
  plt.tight_layout()
  plt.show()


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
files = glob.glob("gdrive/My Drive/tf_video/frames/*")
files.sort()
print(files)

['gdrive/My Drive/tf_video/frames/frame_00001.png', 'gdrive/My Drive/tf_video/frames/frame_00002.png', 'gdrive/My Drive/tf_video/frames/frame_00003.png', 'gdrive/My Drive/tf_video/frames/frame_00004.png', 'gdrive/My Drive/tf_video/frames/frame_00005.png', 'gdrive/My Drive/tf_video/frames/frame_00006.png', 'gdrive/My Drive/tf_video/frames/frame_00007.png', 'gdrive/My Drive/tf_video/frames/frame_00008.png', 'gdrive/My Drive/tf_video/frames/frame_00009.png', 'gdrive/My Drive/tf_video/frames/frame_00010.png', 'gdrive/My Drive/tf_video/frames/frame_00011.png', 'gdrive/My Drive/tf_video/frames/frame_00012.png', 'gdrive/My Drive/tf_video/frames/frame_00013.png', 'gdrive/My Drive/tf_video/frames/frame_00014.png', 'gdrive/My Drive/tf_video/frames/frame_00015.png', 'gdrive/My Drive/tf_video/frames/frame_00016.png', 'gdrive/My Drive/tf_video/frames/frame_00017.png', 'gdrive/My Drive/tf_video/frames/frame_00018.png', 'gdrive/My Drive/tf_video/frames/frame_00019.png', 'gdrive/My Drive/tf_video/fram

In [0]:
images = []
for i in range(8):
    images.append(rescale(io.imread(files[i]).astype(float) / 255.0, (.25, .25, 1)))
image_array = np.concatenate(np.array(images), 2)

In [0]:
data_image_array = image_array.reshape(1, 512, 1024, 24)
full_data_array = [image_array]
for i in range(20):
  full_data_array.append(image_array)
full_data_array = np.array(full_data_array)

In [0]:
(input_layer, encoder_last_layer, decoder_input, decoder_last_layer) = BuildModel(image_array.shape, 1)
full_model = tf.keras.models.Model(input_layer, decoder_last_layer)
opt = keras.optimizers.Adam(lr=.00001)
full_model.compile(optimizer=opt, loss=EvaluatePSNR)
full_model.fit(full_data_array, full_data_array, epochs=400, batch_size=1)

Epoch 1/400
21/21 [==============================] - 1s 70ms/step - loss: 38.4536
Epoch 2/400
21/21 [==============================] - 1s 69ms/step - loss: 38.3877
Epoch 3/400
21/21 [==============================] - 1s 68ms/step - loss: 38.2972
Epoch 4/400
21/21 [==============================] - 1s 69ms/step - loss: 38.1699
Epoch 5/400
21/21 [==============================] - 2s 72ms/step - loss: 37.9857
Epoch 6/400
21/21 [==============================] - 2s 72ms/step - loss: 37.7151
Epoch 7/400
21/21 [==============================] - 1s 71ms/step - loss: 37.3194
Epoch 8/400
21/21 [==============================] - 1s 71ms/step - loss: 36.7545
Epoch 9/400
21/21 [==============================] - 1s 69ms/step - loss: 36.0276
Epoch 10/400
21/21 [==============================] - 1s 70ms/step - loss: 35.3362
Epoch 11/400
21/21 [==============================] - 1s 70ms/step - loss: 34.9188
Epoch 12/400
21/21 [==============================] - 1s 70ms/step - loss: 34.6100
Epoch 13/400


In [0]:
output = full_model.predict(data_image_array)

psnr = tf.image.psnr(data_image_array[0], output[0], max_val=1.0)
print(psnr.numpy())

PlotFrames(data_image_array[0], output[0])